In [3]:
!pip install selenium bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 48.2 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [selenium]/11 [selenium]


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# driver = webdriver.Chrome()
# driver.get("https://www.agnoshealth.com/forums/search")
# driver.implicitly_wait(30)

# postal_offices = []

# # Keep clicking the "Next" button until there are no more pages
# while True:
#     page_source = driver.page_source
#     soup = BeautifulSoup(page_source, "html.parser")
#     tds = soup.find_all('td', {'onclick': True})
#     for td in tds:
#         onclick_text = td['onclick']
#         match = re.search(r'\{.*\}', onclick_text)
#         if match:
#             json_string = match.group()
#             data = json.loads(json_string)
#             name = data['name']
#             location_office = data['location_office']
#             if 'geolocation' in data and data['geolocation'] is not None:
#                 geolocation = data['geolocation']
#                 lat, long = geolocation['lat'], geolocation['long']
#                 geolocation = (lat, long)
#             else:
#                 geolocation = "NA"
#             postal_offices.append({
#                 'name': name,
#                 'location_office': location_office,
#                 'geolocation':geolocation
#             })
#     try:
#         next_button = WebDriverWait(driver, 30).until(
#             EC.element_to_be_clickable((By.XPATH, "//a[@class='page-link'][contains(text(),'ถัดไป')]"))
#         )
#         next_button.click()
#     except:
#         break



In [13]:
driver = webdriver.Chrome()
driver.get("https://www.agnoshealth.com/forums/search")

time.sleep(10)  # wait for page to load

data = []
page = 1

while True:
    print(f"Scraping page {page}...")
    
    # Wait for cards to load
    try:
        cards = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, "article"))
        )
    except:
        print("⚠️ No cards found on this page, stopping.")
        break

    # Extract each card
    for card in cards:
        try:
            gender_age_text = card.find_element(By.CSS_SELECTOR, "p.text-sm.text-gray-500").text
        except:
            gender_age_text = None

        # Default values
        gender, age = None, None
        if gender_age_text:
            # Example: "หญิง | อายุ 23 ปี"
            parts = gender_age_text.split("|")
            if len(parts) >= 2:
                gender = parts[0].strip()
                age_match = re.search(r"อายุ\s*(\d+)", parts[1])
                if age_match:
                    age = int(age_match.group(1))

        try:
            title = card.find_element(By.CSS_SELECTOR, "p.font-bold").text
        except:
            title = None
        try:
            date = card.find_element(By.TAG_NAME, "time").text
        except:
            date = None
        try:
            badges = [b.text for b in card.find_elements(By.CSS_SELECTOR, "ul li")]
        except:
            badges = []
        try:
            detail = card.find_element(By.CSS_SELECTOR, "p.text-sm.text-gray-500.line-clamp-3").text
        except:
            detail = None

        data.append({
            "gender": gender,
            "age": age,
            "title": title,
            "date": date,
            "symptom_badge": ", ".join(badges),
            "detail_symptom": detail
        })

    # Check if page returned no useful data (safety break)
    if len(data) == 0 or all(v is None for v in data[-len(cards):][0].values()):
        print(f"⚠️ Page {page} returned empty data. Stopping.")
        break

    # Try to click "Next Page"
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, "button.bg-primary_blue-500")
        driver.execute_script("arguments[0].click();", next_btn)
        page += 1
        time.sleep(3)  # wait for new page to load
    except:
        print("✅ Last page reached.")
        break

driver.quit()

# Save to CSV
df = pd.DataFrame(data)
df.to_csv("agnos_forums.csv", index=False, encoding="utf-8-sig")

print("Saved", len(df), "records.")

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping page 49...
Scraping page 50...
Scraping 

In [16]:
df

,gender,age,title,date,symptom_badge,detail_symptom,province
0,หญิง,23.0,เสี่ยงต่อโรคซึมเศร้า (Major depressive disorder),2/18/2024,"เสี่ยงต่อโรคซึมเศร้า, อ่อนเพลีย, ไม่มีแรง, +3",สวัสดีค่ะ อยากจะปรึกษาค่ะในช่วง2-3วันที่ผ่านมา...,
1,ชาย,21.0,กระเพาะปัสสาวะอักเสบ (Cystitis),2/16/2024,"กระเพาะปัสสาวะอักเสบ, ปัสสาวะกะปริบกะปรอย, ปัส...",สวัสดีครับคือว่าผมมีอาการนี้มาได้ปีสองปีแล้วคร...,
2,ชาย,20.0,ติดเชื้อชั้นใต้ผิวหนัง (Cellulitis),2/15/2024,"บวมตามร่างกาย, ขาบวม, บวม, +2",ลื่นแล้วท้องไปชนอ่างและมีน้ำใสๆปนเลือดออกจากสะ...,
3,หญิง,20.0,ปวดท้องประจำเดือน (Dysmenorrhea unpecified),11/27/2023,"จุกแน่นท้อง, ปวดท้องประจำเดือน, ท้องอืด, +3",คือมีอะไรกับแฟน วันที่20 ใส่ถุงยางไม่รั่วไม่หล...,
4,ชาย,17.0,แผลริมอ่อน (Chancroid),11/26/2023,แผลริมอ่อน,สวัสดีฮะ เอ่อ..พูดไงดี555พอดีผมมีตุ่มอะไรก้อไม...,
...,...,...,...,...,...,...,...
17539,หญิง,20.0,ปวดท้องประจำเดือน (Dysmenorrhea unpecified),11/27/2023,"จุกแน่นท้อง, ปวดท้องประจำเดือน, ท้องอืด, +3",คือมีอะไรกับแฟน วันที่20 ใส่ถุงยางไม่รั่วไม่หล...,
17540,ชาย,17.0,แผลริมอ่อน (Chancroid),11/26/2023,แผลริมอ่อน,สวัสดีฮะ เอ่อ..พูดไงดี555พอดีผมมีตุ่มอะไรก้อไม...,
17541,ชาย,48.0,เสี่ยงต่อโรคทางอารมณ์ที่มีภาวะแทรกซ้อน (Mood d...,11/23/2023,"คิดฆ่าตัวตาย, เสี่ยงต่อโรคทางอารมณ์ที่มีภาวะแท...",รักษาแบบไม่มีค่าใช้จ่าย มีไหมครับ,
17542,หญิง,18.0,None,None,,None,


In [15]:
provinces = ['เชียงใหม่','แม่ฮ่องสอน', 'เชียงราย' , 'ลำพูน' , 'ลำปาง' , 'พะเยา' , 'แพร่' , 'น่าน' , 'อุตรดิตถ์' , 'ตาก' , 'สุโขทัย' , 'พิษณุโลก' , 'กำแพงเพชร' , 'เพชรบูรณ์' , 'พิจิตร'
             'กาฬสินธุ์' , 'ขอนแก่น' , 'ชัยภูมิ' , 'นครพนม' , 'นครราชสีมา' , 'บุรีรัมย์' , 'มหาสารคาม' , 'มุกดาหาร' , 'ยโสธร', 'ร้อยเอ็ด' , 'เลย' , 'ศรีสะเกษ' , 'สกลนคร' , 'สุรินทร์' 
             , 'หนองคาย' , 'หนองบัวลำภู' , 'อุดรธานี' , 'อุบลราชธานี' , 'อำนาจเจริญ','กระบี่','ชุมพร','ตรัง','นครศรีธรรมราช','นราธิวาส','ปัตตานี','พังงา','พัทลุง','ภูเก็ต','ยะลา'
             ,'ระนอง','สงขลา','สตูล','สุราษฎร์ธานี','กรุงเทพมหานคร','กทม.', 'อุทัยธานี' , 'ชัยนาท' , 'นครสวรรค์' , 'นนทบุรี' , 'ปทุมธานี' , 'พระนครศรีอยุธยา' , 'ลพบุรี' , 'สมุทรปราการ'
            , 'สมุทรสงคราม' , 'สมุทรสาคร' , 'สระบุรี' , 'สิงห์บุรี' , 'อ่างทอง', 'ชลบุรี' , 'ระยอง' , 'จันทบุรี' , 'ตราด' , 'นครนายก' , 'ฉะเชิงเทรา' , 'ปราจีนบุรี' , 'สระแก้ว',
            'กาญจนบุรี' , 'ประจวบคีรีขันธ์' , 'สุพรรณบุรี' , 'เพชรบุรี' , 'นครปฐม' , 'ราชบุรี'] # list of all provinces
df['province'] = ''
for i in range(len(df)):
    for province in provinces:
        if province in df.loc[i, 'location_office']:
            df.loc[i, 'province'] = province
            break


KeyError: 'location_office'

In [ ]:
df = df.replace('กทม.','กรุงเทพมหานคร')

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.shape

(1380, 4)

In [ ]:
df.head()

,name,geolocation,province
0,คปณ. Botany,"(14.064928799362482, 100.64406752586365)",ปทุมธานี
2,คปณ. Challenger,"(13.91439, 100.54791)",นนทบุรี
4,คปณ. Garden Watladpladuk,"(13.907871485140404, 100.39540797472)",นนทบุรี
6,คปณ.Bangkok Mediplex (ณุศาศิริแกรนด์คอนโด),"(13.719085064608725, 100.58451443910599)",กรุงเทพมหานคร
8,คปณ.GCS เชียงใหม่,NA,เชียงใหม่


In [ ]:
df.drop('location_office',axis=1,inplace=True)

In [ ]:
from pathlib import Path  
filepath = Path('/Users/chai/Desktop/Intern/Python/Q3.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath) 